In [1]:
import sys
sys.path.append('/content')
from src.fluvius import USGS_Water_DB

%load_ext autoreload
%autoreload 2

db = USGS_Water_DB()
db.get_station_df()
db.station_df



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
There is no [linux64] chromedriver for browser 90.0.4430 in cache
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Driver has been saved in cache [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24]


,site_no,site_name,Latitude,Longitude,geometry
0,01472157,"French Creek near Phoenixville, PA",40.1515,-75.6013,POINT (-75.60130 40.15150)
1,01478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77080 39.74750)
2,01480617,"West Branch Brandywine Creek at Modena, PA",39.9618,-75.8013,POINT (-75.80130 39.96180)
3,01480870,East Branch Brandywine Creek below Downingtown...,39.9687,-75.6733,POINT (-75.67330 39.96870)
4,01481000,"Brandywine Creek at Chadds Ford, PA",39.8698,-75.5933,POINT (-75.59330 39.86980)
...,...,...,...,...,...
160,393806095273700,"Atchison County Lake near Horton, KS",39.6350,-95.4603,POINT (-95.46030 39.63500)
161,393817095260100,"Clear Creek at Decator Road near Horton, KS",39.6381,-95.4336,POINT (-95.43360 39.63810)
162,394126096073500,Black Vermillion River Tributary above Central...,39.6906,-96.1264,POINT (-96.12640 39.69060)
163,394146096085500,"Centralia Lake near Centralia, KS",39.6961,-96.1486,POINT (-96.14860 39.69610)


In [1]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [2]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

container = 'usgs-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/station_metadata.csv'
station_df = pd.read_csv(station_url, storage_options=storage_options) 

img_station_url = f'az://{container}/station_img_metadata.csv'
img_df = pd.read_csv(img_station_url, storage_options=storage_options)

img_df

In [8]:
img_df

,site_no,site_name,Latitude,Longitude,geometry,cloudless_img_count,date_range
0,1632900,"Smith Creek Near New Market, VA",38.6935,-78.6428,"POLYGON ((-78.63279999999999 38.7035, -78.6327...",100,2015-11-16/2019-12-30
1,1645704,"Difficult Run Above Fox Lake Near Fairfax, VA",38.8847,-77.3324,"POLYGON ((-77.3224 38.8947, -77.3224 38.8747, ...",83,2015-08-05/2019-12-30
2,1645762,S F Little Difficult Run Above Mouth NR Vienna...,38.9089,-77.3383,"POLYGON ((-77.3283 38.9189, -77.3283 38.8989, ...",86,2015-08-05/2019-12-30
3,1646000,"Difficult Run Near Great Falls, VA",38.9759,-77.2458,"POLYGON ((-77.2358 38.9859, -77.2358 38.9659, ...",156,2015-08-05/2019-12-30
4,1646305,"Dead Run at Whann Avenue Near Mclean, VA",38.9598,-77.1757,"POLYGON ((-77.1657 38.9698, -77.1657 38.9498, ...",31,2015-08-05/2016-12-30
5,1649190,"Paint Branch Near College Park, MD",39.0331,-76.9643,"POLYGON ((-76.95429999999999 39.0431, -76.9542...",72,2015-08-05/2019-12-25
6,1649500,"Northeast Branch Anacostia River at Riverdale, MD",38.9603,-76.9260,"POLYGON ((-76.916 38.97029999999999, -76.916 3...",24,2015-08-05/2017-12-30
7,1654000,"Accotink Creek Near Annandale, VA",38.8129,-77.2283,"POLYGON ((-77.2183 38.8229, -77.2183 38.8029, ...",277,2016-04-01/2019-12-30
8,1656903,Flatlick Branch Above Frog Branch at Chantilly...,38.8824,-77.4319,POLYGON ((-77.42189999999999 38.89239999999999...,147,2015-08-05/2019-12-30
9,1673000,"Pamunkey River Near Hanover, VA",37.7676,-77.3322,"POLYGON ((-77.3222 37.7776, -77.3222 37.7576, ...",101,2015-07-26/2019-12-25


## 500 mb per 3 channel image on sentinel-2 including scl downsampled

In [12]:
total_tb = np.sum(img_df.cloudless_img_count)*500/1000000

In [17]:
print(f'USGS total memory needed for {np.sum(img_df.cloudless_img_count)} images: {total_tb} TB')

USGS total memory needed for 3564 images: 1.782 TB


In [18]:
#this does not include prediction

In [21]:
ana = 29*500/1000000

In [23]:
total_tb + ana

1.7965